In [2]:
# -*- coding: utf-8 -*-

# ライブラリインポート

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import os
import csv
import collections
from collections import Counter
from tqdm import tqdm
import re
import itertools

# データの傾向を確認

In [5]:
# df_all_fixの読み込み
df_all_fix_1 = pd.read_csv("../output/df_csv/df_all_fix_1.csv", encoding = "utf-8")
df_all_fix_2 = pd.read_csv("../output/df_csv/df_all_fix_2.csv", encoding = "utf-8")
print(df_all_fix_1.columns)
print(df_all_fix_1.shape)
print(df_all_fix_2.columns)
print(df_all_fix_2.shape)
df_all_fix = df_all_fix_1.append(df_all_fix_2, ignore_index=True)
df_all_fix = df_all_fix.reset_index(drop=True)
# 不要列「Unnamed: 0」を削除
df_all_fix = df_all_fix.ix[:, 1:]
# データ確認
print(df_all_fix.shape)
df_all_fix.head(10)

Index(['Unnamed: 0', 'レンタルタイプ', '都道府県', '市区町村', '町域以下', '立地1_路線', '立地1_駅',
       '立地1_バス', '立地1_徒歩', '立地2_路線', '立地2_駅', '立地2_バス', '立地2_徒歩', '立地3_路線',
       '立地3_駅', '立地3_バス', '立地3_徒歩', '築年数', '地下', '地上', '階', '賃料', '管理費', '敷金',
       '礼金', '保証金', '敷引,償却費', '部屋数', 'D有無', 'K有無', 'L有無', 'S有無', '専有面積',
       '月額費用', '初期費用'],
      dtype='object')
(300001, 35)
Index(['Unnamed: 0', 'レンタルタイプ', '都道府県', '市区町村', '町域以下', '立地1_路線', '立地1_駅',
       '立地1_バス', '立地1_徒歩', '立地2_路線', '立地2_駅', '立地2_バス', '立地2_徒歩', '立地3_路線',
       '立地3_駅', '立地3_バス', '立地3_徒歩', '築年数', '地下', '地上', '階', '賃料', '管理費', '敷金',
       '礼金', '保証金', '敷引,償却費', '部屋数', 'D有無', 'K有無', 'L有無', 'S有無', '専有面積',
       '月額費用', '初期費用'],
      dtype='object')
(323993, 35)
(623994, 34)


,レンタルタイプ,都道府県,市区町村,町域以下,立地1_路線,立地1_駅,立地1_バス,立地1_徒歩,立地2_路線,立地2_駅,...,保証金,"敷引,償却費",部屋数,D有無,K有無,L有無,S有無,専有面積,月額費用,初期費用
0,賃貸アパート,東京,足立,西綾瀬４,東京メトロ千代田線,綾瀬駅,0,11,つくばエクスプレス,青井駅,...,0.0,0.0,2.0,0.0,1.0,0.0,0.0,24.00,26000.0,50000.0
1,賃貸アパート,東京,足立,小台２,日暮里・舎人ライナー,足立小台駅,0,17,ＪＲ高崎線,尾久駅,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11.57,26000.0,52000.0
2,賃貸アパート,東京,足立,新田１,東京メトロ南北線,王子神谷駅,0,15,-,-,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,12.00,26500.0,50000.0
3,賃貸アパート,東京,足立,新田１,東京メトロ南北線,王子神谷駅,0,17,-,-,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.01,26500.0,50000.0
4,賃貸アパート,東京,足立,新田１,東京メトロ南北線,王子神谷駅,0,17,-,-,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.01,26500.0,50000.0
5,賃貸アパート,東京,足立,梅島２,東武伊勢崎線,梅島駅,0,9,東武伊勢崎線,西新井駅,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.00,28000.0,28000.0
6,賃貸アパート,東京,足立,梅島２,東武伊勢崎線,梅島駅,0,9,東武伊勢崎線,西新井駅,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.00,28000.0,28000.0
7,賃貸アパート,東京,足立,千住曙町,東京メトロ千代田線,北千住駅,0,15,京成本線,京成関屋駅,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.00,28000.0,56000.0
8,賃貸アパート,東京,足立,千住曙町,東京メトロ千代田線,北千住駅,0,15,東武伊勢崎線,牛田駅,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.00,28000.0,56000.0
9,賃貸アパート,東京,足立,千住曙町,東京メトロ千代田線,北千住駅,0,15,東武伊勢崎線,牛田駅,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.00,28000.0,56000.0


まず、23区の物件数の傾向を見てみる。

# 予測モデル作成

# お買い物物件を探す